In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.spatial.distance import pdist
from sklearn.cluster import KMeans

In [4]:
# optimized partitional models
clara_opt = pd.read_csv('../2_models/with_raw_data/solar/results/partitional_opt/csv_labels_raw/clara.csv')
fuzzy_opt = pd.read_csv('../2_models/with_raw_data/solar/results/partitional_opt/csv_labels_raw/fuzzy_c_means.csv')
kmeans_opt = pd.read_csv('../2_models/with_raw_data/solar/results/partitional_opt/csv_labels_raw/kmeans.csv')

# partitionalal models
clara = pd.read_csv('../2_models/with_raw_data/solar/results/partitional/csv_labels_raw/clara.csv')
fuzzy = pd.read_csv('../2_models/with_raw_data/solar/results/partitional/csv_labels_raw/fuzzy_c_means.csv')
kmeans = pd.read_csv('../2_models/with_raw_data/solar/results/partitional/csv_labels_raw/kmeans.csv')

# model based
bgmm = pd.read_csv('../2_models/with_raw_data/solar/results/model_based/csv_labels_cut_raw/bgmm.csv')
# dpmm = pd.read_csv('../2_models/with_raw_data/solar/results/model_based/csv_labels_cut_raw/dpmm.csv')
gaussian_mix = pd.read_csv('../2_models/with_raw_data/solar/results/model_based/csv_labels_cut_raw/gaussian_mix.csv')

# optimized model based
bgmm_opt = pd.read_csv('../2_models/with_raw_data/solar/results/model_based_opt/csv_labels_cut_raw/bgmm.csv')
gaussian_mix_opt = pd.read_csv('../2_models/with_raw_data/solar/results/model_based_opt/csv_labels_cut_raw/gaussian_mix.csv')

# optimized hierarchical models
average_link_opt = pd.read_csv('../2_models/with_raw_data/solar/results/hierarchical_opt/csv_labels_cut_raw/average_link.csv')
birch_opt = pd.read_csv('../2_models/with_raw_data/solar/results/hierarchical_opt/csv_labels_cut_raw/birch.csv')
single_link_opt = pd.read_csv('../2_models/with_raw_data/solar/results/hierarchical_opt/csv_labels_cut_raw/single_link.csv')
centroid_link_opt = pd.read_csv('../2_models/with_raw_data/solar/results/hierarchical_opt/csv_labels_cut_raw/centroid_link.csv')
ward_link_opt = pd.read_csv('../2_models/with_raw_data/solar/results/hierarchical_opt/csv_labels_cut_raw/ward_link.csv')

# hierarchical models
average_link = pd.read_csv('../2_models/with_raw_data/solar/results/hierarchical/csv_labels_cut_raw/average_link.csv')
birch = pd.read_csv('../2_models/with_raw_data/solar/results/hierarchical/csv_labels_cut_raw/birch.csv')
single_link = pd.read_csv('../2_models/with_raw_data/solar/results/hierarchical/csv_labels_cut_raw/single_link.csv')
centroid_link = pd.read_csv('../2_models/with_raw_data/solar/results/hierarchical/csv_labels_cut_raw/centroid_link.csv')
ward_link = pd.read_csv('../2_models/with_raw_data/solar/results/hierarchical/csv_labels_cut_raw/ward_link.csv')

# density based models
dbscan = pd.read_csv('../2_models/with_raw_data/solar/results/density_based/csv_labels_cut_raw/DBSCAN.csv')
optics = pd.read_csv('../2_models/with_raw_data/solar/results/density_based/csv_labels_cut_raw/OPTICS.csv')

In [5]:
dbscan

,0,1,2,3,4,cluster_label,asset_id
0,-0.508250,-0.893884,-2.667186,0.163522,-0.072287,1,11
1,-0.469440,-0.605656,-1.540451,0.408095,-0.780179,1,11
2,-0.462222,-0.566264,-1.417633,-0.185869,0.300288,1,11
3,-0.499687,-0.826978,-1.690806,-0.360564,1.902361,1,11
4,-0.211838,1.313884,-1.545407,-1.059346,-0.705664,1,11
...,...,...,...,...,...,...,...
40112,5.307196,0.175032,0.412244,0.512912,-0.780179,1,514
40113,4.591347,0.062674,0.279709,1.036998,-0.780179,1,514
40114,-0.303719,-0.856431,0.299107,-0.011174,-0.370347,1,514
40115,-0.411631,-0.876188,0.199798,-0.011174,-0.780179,1,514


In [6]:
from scipy.spatial.distance import pdist, cdist

def dunn_index(df):
    points = df[['0', '1', '2', '3', '4']].values
    labels = df['cluster_label'].values
    unique_labels = np.unique(labels)
    
    max_intra_cluster_distances = []
    min_inter_cluster_distance = float('inf')

    for label in unique_labels:
        cluster_points = points[labels == label]
        if len(cluster_points) > 1:
            intra_distances = pdist(cluster_points)
            max_intra_cluster_distances.append(np.max(intra_distances))
        else:
            max_intra_cluster_distances.append(0)

    for i, label_i in enumerate(unique_labels):
        cluster_points_i = points[labels == label_i]
        for j, label_j in enumerate(unique_labels):
            if label_i < label_j:
                cluster_points_j = points[labels == label_j]
                inter_distances = cdist(cluster_points_i, cluster_points_j)
                min_inter_cluster_distance = min(min_inter_cluster_distance, np.min(inter_distances))

    if len(max_intra_cluster_distances) == 0 or min_inter_cluster_distance == float('inf'):
        return 0

    return min_inter_cluster_distance / max(max_intra_cluster_distances)

In [7]:
# def xie_beni_index(df):
#     # Compute the total scatter
#     total_scatter = np.sum(pdist(df[['0', '1', '2', '3' , '4']]) ** 2) / (2 * len(df))
    
#     # Compute the intra-cluster scatter
#     cluster_labels = df['cluster_label'].unique()
#     intra_cluster_scatter = 0
#     for label in cluster_labels:
#         cluster_points = df[df['cluster_label'] == label][['0', '1', '2', '3' , '4']]
#         cluster_center = np.mean(cluster_points, axis=0)
#         intra_cluster_scatter += np.sum(np.linalg.norm(cluster_points - cluster_center, axis=1) ** 2)
#     intra_cluster_scatter /= len(df)
#     return intra_cluster_scatter / total_scatter

In [8]:
# Define a function to compute evaluation metrics
def compute_metrics(df):
    metrics = {}
    
    if df['cluster_label'].nunique() > 1:
        metrics['Silhouette Score'] = silhouette_score(df[['0', '1', '2', '3' , '4']], df['cluster_label'])
        metrics['Davies-Bouldin Index'] = davies_bouldin_score(df[['0', '1', '2', '3' , '4']], df['cluster_label'])
        metrics['Calinski-Harabasz Index'] = calinski_harabasz_score(df[['0', '1', '2', '3' , '4']], df['cluster_label'])
        metrics['Dunn Index'] = dunn_index(df)
        # metrics['Xie-Beni Index'] = xie_beni_index(df)
    else:
        metrics['Silhouette Score'] = 0
        metrics['Davies-Bouldin Index'] = 0
        metrics['Calinski-Harabasz Index'] = 0
        metrics['Dunn Index'] = 0
        # metrics['Xie-Beni Index'] = 0

    return metrics


In [9]:
dataframes = {
    'clara_opt': clara_opt, 'fuzzy_opt': fuzzy_opt, 'kmeans_opt': kmeans_opt,
    'clara': clara, 'fuzzy': fuzzy, 'kmeans': kmeans,
    'bgmm': bgmm, 'gaussian_mix': gaussian_mix,
    'average_link_opt': average_link_opt, 'birch_opt': birch_opt,
    'single_link_opt': single_link_opt, 'centroid_link_opt': centroid_link_opt,
    'ward_link_opt': ward_link_opt,
    'average_link': average_link, 'birch': birch, 'single_link': single_link,
    'centroid_link': centroid_link, 'ward_link': ward_link,
    'dbscan': dbscan, 'optics': optics,
    'bgmm_optimized': bgmm_opt, 'gaussian_mix_opt' : gaussian_mix_opt
 }

In [10]:
# Iterate over each dataframe and print the name of the ones without a 'cluster_label' column
for model_name, df in dataframes.items():
    if 'cluster_label' not in df.columns:
        print(f"The dataframe '{model_name}' does not have a column named 'cluster_label'.")
        print(df.columns)

The dataframe 'fuzzy_opt' does not have a column named 'cluster_label'.
Index(['0', '1', '2', '3', '4', 'cluster', 'asset_id'], dtype='object')
The dataframe 'kmeans_opt' does not have a column named 'cluster_label'.
Index(['0', '1', '2', '3', '4', 'cluster', 'asset_id'], dtype='object')
The dataframe 'clara' does not have a column named 'cluster_label'.
Index(['0', '1', '2', '3', '4', 'cluster', 'asset_id'], dtype='object')
The dataframe 'fuzzy' does not have a column named 'cluster_label'.
Index(['0', '1', '2', '3', '4', 'cluster', 'asset_id'], dtype='object')
The dataframe 'kmeans' does not have a column named 'cluster_label'.
Index(['0', '1', '2', '3', '4', 'cluster', 'asset_id'], dtype='object')


In [11]:
# Loop through each dataframe and rename the 'cluster' column to 'cluster_label'
for model_name, df in dataframes.items():
    if 'cluster' in df.columns:
        df.rename(columns={'cluster': 'cluster_label'}, inplace=True)

In [12]:
# Initialize an empty list to store the results
results = []

# Compute and store the evaluation metrics for each dataframe
for model_name, df in dataframes.items():
    print(f'Evaluating {model_name}')
    metrics = compute_metrics(df)
    results.append({'Model': model_name, **metrics})

# Create a DataFrame from the results list
results_df = pd.DataFrame(results)

Evaluating clara_opt


Evaluating fuzzy_opt
Evaluating kmeans_opt
Evaluating clara
Evaluating fuzzy
Evaluating kmeans
Evaluating bgmm
Evaluating gaussian_mix
Evaluating average_link_opt
Evaluating birch_opt
Evaluating single_link_opt
Evaluating centroid_link_opt
Evaluating ward_link_opt
Evaluating average_link
Evaluating birch
Evaluating single_link
Evaluating centroid_link
Evaluating ward_link
Evaluating dbscan
Evaluating optics
Evaluating bgmm_optimized
Evaluating gaussian_mix_opt


In [13]:
results_df.to_csv('results/raw_solar.csv', index=False)

In [14]:
results_df

,Model,Silhouette Score,Davies-Bouldin Index,Calinski-Harabasz Index,Dunn Index
0,clara_opt,0.226465,1.478942,10618.282648,0.000954
1,fuzzy_opt,0.161198,2.155985,7699.307718,0.001138
2,kmeans_opt,0.330920,1.047904,15456.711037,0.001360
3,clara,0.235793,1.236165,13367.366516,0.000755
4,fuzzy,0.226489,1.812524,8296.976576,0.000696
5,kmeans,0.328741,1.004836,17873.527540,0.001309
6,bgmm,0.210260,1.793964,5717.261203,0.000571
7,gaussian_mix,0.208937,1.802679,5688.439922,0.000811
8,average_link_opt,0.445281,0.647322,2135.307590,0.023271
9,birch_opt,0.254211,1.439677,6645.114894,0.001111


In [15]:
results_df = pd.read_csv('results/raw_solar.csv')

In [16]:
# Convert columns to numeric data types
normalized_df = results_df.copy()
for col in normalized_df.columns[1:]:
    normalized_df[col] = pd.to_numeric(normalized_df[col], errors='coerce')

# Drop rows with missing or non-numeric values
normalized_df = normalized_df.dropna()

# Define ideal and anti-ideal solutions for each metric
ideal_solution = normalized_df.copy()
anti_ideal_solution = normalized_df.copy()

# Define metrics where higher values are better
higher_is_better = ['Silhouette Score', 'Dunn Index', 'Calinski-Harabasz Index']

# Define metrics where lower values are better
lower_is_better = ['Davies-Bouldin Index']

# Set ideal and anti-ideal solutions for metrics where higher values are better
for metric in higher_is_better:
    ideal_solution[metric] = normalized_df[metric].max()
    anti_ideal_solution[metric] = normalized_df[metric].min()

# Set ideal and anti-ideal solutions for metrics where lower values are better
for metric in lower_is_better:
    ideal_solution[metric] = normalized_df[metric].min()
    anti_ideal_solution[metric] = normalized_df[metric].max()

# Calculate the distance from each alternative to the ideal and anti-ideal solutions
dist_to_ideal = ((normalized_df.iloc[:,1:] - ideal_solution) ** 2).sum(axis=1) ** 0.5
dist_to_anti_ideal = ((normalized_df.iloc[:,1:] - anti_ideal_solution) ** 2).sum(axis=1) ** 0.5

# Calculate the relative closeness to the ideal solution
normalized_df['Closeness'] = dist_to_anti_ideal / (dist_to_ideal + dist_to_anti_ideal)

# Sort the models based on their relative closeness to the ideal solution
ranked_models = normalized_df.sort_values(by='Closeness', ascending=False)

# Print the ranked models
ranked_models[['Model', 'Closeness']]

,Model,Closeness
5,kmeans,0.999953
2,kmeans_opt,0.864623
17,ward_link,0.772378
12,ward_link_opt,0.772378
3,clara,0.747589
0,clara_opt,0.593600
14,birch,0.515019
4,fuzzy,0.463573
1,fuzzy_opt,0.430095
20,bgmm_optimized,0.384085


3 best models: kmeans, clara, ward

(not including kmeans_opt since the best model is kmeans)

In [18]:
results_df.iloc[[5, 17, 3]]

,Model,Silhouette Score,Davies-Bouldin Index,Calinski-Harabasz Index,Dunn Index
5,kmeans,0.328741,1.004836,17873.527540,0.001309
17,ward_link,0.257280,1.099358,13809.903607,0.004169
3,clara,0.235793,1.236165,13367.366516,0.000755
